In [2]:
# TODO : generator
from lib.ekyn import *
from sage.utils import *
from sage.models import *
from time import time
import datetime
import copy
import os
from sage.models import Dumbledore
from lib.ekyn import *
from sage.utils import count_params

experiment_group_id = 'lstm'
hyperparameters = {
    'wd':1e-2,
    'lr':3e-4,
    'batch_size':2048,
    'robust':False,
    'norm':'batch',
    'dropout':.1,
    'stem_kernel_size':3,
    'widthi':[4,8,16,32],
    'depthi':[1,1,1,1],
    'patience':100,
    'scheduler_patience':50,
    'epochs':500,
    'sequence_length':21,
    'encoder_experiment_name':f'2024_14_08_16_04_12',
    'hidden_size':64,
    'num_layers':1,
    'frozen_encoder':True
}

trainloader,testloader = get_sequenced_dataloaders(batch_size=hyperparameters['batch_size'],sequence_length=hyperparameters['sequence_length'])
model = Dumbledore(encoder_experiment_name=hyperparameters['encoder_experiment_name'],sequence_length=hyperparameters['sequence_length'],hidden_size=hyperparameters['hidden_size'],num_layers=hyperparameters['num_layers'],dropout=hyperparameters['dropout'],frozen_encoder=hyperparameters['frozen_encoder'])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=hyperparameters['lr'],weight_decay=hyperparameters['wd'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=hyperparameters['scheduler_patience'])

state = {
    'start_time':datetime.datetime.now().strftime("%Y_%d_%m_%H_%M_%S"),
    'experiment_group_id':experiment_group_id,
    'execution_time':0,
    'device':'cuda',
    'trainlossi':[],
    'testlossi':[],
    'best_dev_loss':torch.inf,
    'model':model,
    'scheduler':scheduler,
    'criterion':criterion,
    'optimizer':optimizer,
    'best_model_wts':copy.deepcopy(model.state_dict()),
}

os.makedirs(f'experiments/{state["start_time"]}')

for key in hyperparameters:
    state[key] = hyperparameters[key]

last_time = time()

for state in train(state,trainloader,testloader):
    plot_loss(state)
    state['execution_time'] = (state['execution_time'] + (time() - last_time))/2
    last_time = time()
    torch.save(state, f'experiments/{state["start_time"]}/state.pt')
torch.save(state, f'experiments/{state["start_time"]}/state.pt')

/home/andrew/sleep/sleep_ml/sage/models.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f'experiments/{encoder_experiment_name}/state.pt',map_locatio

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/2024_14_08_16_04_12/state.pt'

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,f1_score

best_model = copy.deepcopy(state['model'])
best_model.load_state_dict(state['best_model_wts'])
loss,y_true,y_pred = evaluate(dataloader=testloader,model=best_model,criterion=criterion,device='cuda')
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',colorbar=False)
plt.title(f'f1 : {f1_score(y_true,y_pred,average="macro"):.3f}')

In [ ]:

trainloader,testloader = get_dataloaders(train_data,test_data)
loss,y_true,y_pred = evaluate_sigmoid(dataloader=testloader,model=best_model,criterion=state['criterion'],device='cuda')
f1 = f1_score(y_true,y_pred,average="macro")
print(f1)
df.loc[experiment.name,'best_dev_f1'] = f1

In [ ]:
from sklearn.model_selection import train_test_split
from lib.ekyn import *

ekyn_ids = get_ekyn_ids()
train_ids,test_ids = train_test_split(ekyn_ids,test_size=.2,shuffle=True,random_state=0)
print(train_ids,test_ids)

In [ ]:
for id in test_ids:
    dataloader = get_epoched_dataloader_for_ids(ids=[id])
    loss,y_true,y_pred = evaluate(dataloader=dataloader,model=model,criterion=criterion,device='cuda')
    ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',colorbar=False)
    plt.title(f'f1 : {f1_score(y_true,y_pred,average="macro"):.3f}')

In [ ]:

fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(8,4))
loss,y_true,y_pred = evaluate(dataloader=trainloader,model=model,criterion=criterion,device='cuda')
print(loss)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',ax=axes[0],colorbar=False)
axes[0].set_title(f'trainf1 : {f1_score(y_true,y_pred,average="macro"):.3f}')
loss,y_true,y_pred = evaluate(dataloader=testloader,model=model,criterion=criterion,device='cuda')
print(loss)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',ax=axes[1],colorbar=False)
axes[1].set_title(f'testf1 : {f1_score(y_true,y_pred,average="macro"):.3f}')

In [ ]:
df = pd.DataFrame()

plt.figure(figsize=(14,8))
for id in test_ids:
    X,y = load_ekyn_pt_robust(id=id,condition='Vehicle',downsampled=True)
    std,mean = torch.std_mean(X[torch.where(y.argmax(axis=1) == 1)[0]],dim=1)
    sns.ecdfplot(std,linewidth=2,label=id)

# df_long = df.melt()
# df_long[['id','condition']] = df_long.variable.str.split('_',expand=True)
# sns.ecdfplot(df_long[df_long.condition == 'Vehicle'],x='value',hue='id',linestyle='-')
# sns.ecdfplot(df_long[df_long.condition == 'PF'],x='value',hue='id',linestyle='--')
# std,mean = torch.std_mean(eeg,dim=1)

plt.legend()
plt.xlim([0,2])